# Linear Eigenvalue Problems

In [2]:
import numpy as np
import matplotlib.pyplot as plt

Let's consider a symmetric, square matrix $\mathbf{A}$. An eigenvector $\mathbf{v}$ is a vector that satisfies

$$\mathbf{A} \mathbf{v} = \lambda \mathbf{v}$$

where $\lambda$ is the corresponding eigenvalue. For an $N \times N$ matrix there are $N$ eigenvectors $\mathbf{v}_1 ... \mathbf{v}_N$ with eigenvalues $\lambda_1 ... \lambda_N$. How do we go about solving this type of linear eigenvalue problem? First let's realize that we can consider the eigenvectors to be the columns of a single $N \times N$ matrix $\mathbf{V}$, which lets us combine all equations $\mathbf{A} \mathbf{v_i} = \lambda_i \mathbf{v_i}$ into one matrix equation,

$$\mathbf{A} \mathbf{V} = \mathbf{V}\mathbf{D}$$

where $\mathbf{D}$ is the diagonal matrix (nonzero only on the diagonal) with the eigenvalues $\lambda_i$ as its diagonal entries. Thus what we want to do is find $\mathbf{V}$ and $\mathbf{D}$, and we are going to do this by factorizing $\mathbf{A}$ into a product of two matrices with handy properties. 

## The QR Algorithm

In this notebook we will implement a common algorithm for solving eigenvalue problems. We'll compare our by-hand version to the solution we get from numpy's `eig` function. You probably will never again bother to code up your own algorithm for solving eigenvalue problems after this notebook, but it is still a good exercise to go through once, to convince yourself that what functions like `eig` are doing are not all that magical nor mysterious. 

The basic idea is that any real matrix can be decomposed into the form 

$$\mathbf{A} = \mathbf{Q} \cdot \mathbf{R}$$

where $\mathbf{Q}$ is orthogonal (its columns and rows are orthonormal vectors) and $\mathbf{R}$ is an upper-triangular matrix (all the entries below the diagonal are 0). 

The QR algorithm is an iterative algorithm that uses this decomposition to compute the eigenvalues of $\mathbf{A}$. Consider iterations indexed by $k$, where we start with $A_0 = \mathbf{A}$. At the $k^{th}$ iteration, we will compute $A_{k+1} = R_k Q_k$. Then we'll decompose $A_{k+1}$ via the QR decomposition and repeat. The trick here is that $Q_k$ is invertible because its columns are linearly independent, so

$$A_{k+1} = R_k Q_k = Q_k^{-1} A_k Q_k = Q_k^T A_k Q_k$$

Iteratively applying this process reduces $A_k$ to an upper triangular matrix (the Shur decomposition) -- the below-diagonal entries get smaller and smaller with each iteration, and we'll stop when they are below some threshold. The eigenvalues of a triangular matrix are just its diagonal elements, so when we have found that matrix, we'll be done. Because we are going to apply this to a *symmetric* matrix $\textbf{A}$, this is even simpler: the end result will be a diagonal matrix (a symmetric triangular matrix must be diagonal!), where of course the diagonal elements will be the eigenvalues. 

A good linear algebra course will cover more of the theory (proofs) behind this -- today we'll take a practical approach and I will step you through what to do. This will give you more practice with -- you guessed it -- array indexing!

Here is a matrix to use for the following:

In [3]:
A = np.array([[1, 4, 8, 4], 
              [4, 2, 3, 7], 
              [8, 3, 6, 9], 
              [4, 7, 9, 2]], dtype=float)

&#128309; Write a function `mag(v)` that returns the magnitude of a vector `v`, i.e. the square root of the dot product of `v` with itself. ✅ Use `np.dot` to compute the dot product. ✅

In [4]:
def mag(v):
    return np.sqrt(np.dot(v, v))

### The QR decomposition

The trickiest part of the QR algorithm is just doing the QR decomposition of $\mathbf{A}$, so we'll do that first. The algorithm I'll step you through below is doing the following. Think of our matrix $\mathbf{A}$ as being a set of $N$ column vectors $\mathbf{a}_0, \mathbf{a}_1, ... \mathbf{a}_{N-1}$. We're going to define two new sets of vectors $\mathbf{u}_0, \mathbf{u}_1, ... \mathbf{u}_{N-1}$ and $\mathbf{q}_0, \mathbf{q}_1, ... \mathbf{q}_{N-1}$ as follows:

$$\mathbf{u}_0 = \mathbf{a}_0, ~~~~ \mathbf{q}_0 = \frac{\mathbf{u}_0}{\left|\mathbf{u}_0\right|}$$

$$\mathbf{u}_1 = \mathbf{a}_1 - (\mathbf{q}_0 \cdot \mathbf{a}_1)\mathbf{q}_0, ~~~~ \mathbf{q}_1 = \frac{\mathbf{u}_1}{\left|\mathbf{u}_1\right|}$$

$$\mathbf{u}_2 = \mathbf{a}_2 - (\mathbf{q}_0 \cdot \mathbf{a}_2)\mathbf{q}_0 - (\mathbf{q}_1 \cdot \mathbf{a}_2)\mathbf{q}_1, ~~~~ \mathbf{q}_2 = \frac{\mathbf{u}_2}{\left|\mathbf{u}_2\right|}$$

and so on!

In other words, we are defining vectors $\mathbf{u}$ and $\mathbf{q}$ such that 

$$\mathbf{a}_0 = \left|\mathbf{u_0}\right| \mathbf{q}_0,\\
 \mathbf{a}_1 = \left|\mathbf{u_1}\right| \mathbf{q_1} + (\mathbf{q_0}\cdot\mathbf{a_1})\mathbf{q_0},
$$

etc. Then grouping the $\mathbf{q}$ vectors as columns of their own matrix $\mathbf{Q}$, we will have found the QR decomposition. 

&#128309; Write a function `QR_decomposition` that decomposes a matrix A into Q and R matrices, following the comments below. ✅

In [6]:
def QR_decomposition(A):
    # Define N to be the length of one side of the matrix A
    N, _ = A.shape
    # Initialize Q, R, and U matrices of the same shape as A
    Q, R, U = np.zeros((A.shape)), np.zeros((A.shape)), np.zeros((A.shape))
    # Step through each column of A: call this index i
    for i in range(N):
        # Set the current column of U to the current column of A
        U[:, i] = A[:, i]
        # Step through each column of U up to the current column index of A: call this index j
        for j in range(0, i):
            # Set R[j, i] to the dot product of the jth column of Q and the ith column of A
            R[j, i] = np.dot(Q[:, j], A[:, i])
            # Subtract the projection of the jth column of A onto Q from U
            # In other words, subtract R[j, i] * Q[:, j] from U[:, i]
            U[:, i] -= R[j, i] * Q[:, j]

        # Set the current (i-th) column of Q to the current column of U divided by its magnitude
        Q[:, i] = U[:, i] / mag(U[:, i])
        # Set the diagonal element of R to the magnitude of the i-th column of U (compute the magnitude using mag)
        R[i, i] = mag(U[:, i])
    return Q, R

&#128309; Apply `QR_decomposition` to A. ✅ Print the output and verify that Q is an orthogonal matrix and R is an upper-triangular matrix. *Hint:* Recall that for an orthogonal matrix $\mathbf{Q}$, $\mathbf{Q}^T \mathbf{Q} = \mathbb{I}$, where $\mathbb{I}$ is the identity matrix. ✅

In [7]:
Q, R = QR_decomposition(A)

print("Q^T*Q = I = \n", np.dot(Q.T, Q))
print("R = \n", R)

Q^T*Q = I = 
 [[ 1.00000000e+00 -3.39895261e-16  5.36675579e-16 -1.64186462e-15]
 [-3.39895261e-16  1.00000000e+00  1.91661294e-15 -6.92056403e-16]
 [ 5.36675579e-16  1.91661294e-15  1.00000000e+00 -1.74676443e-15]
 [-1.64186462e-15 -6.92056403e-16 -1.74676443e-15  1.00000000e+00]]
R = 
 [[ 9.8488578   6.49821546 10.55960012 11.37187705]
 [ 0.          5.98106979  8.4234836  -0.484346  ]
 [ 0.          0.          2.74586406  3.27671222]
 [ 0.          0.          0.          3.11592335]]


✍🏽 Here we can confirm that Q^T*Q is equal to the identity matrix and that R is upper triangular. 

&#128309; Verify that your code worked by taking the dot product of `Q` and `R`. If you want you can also compare your output to the output of `np.linalg.qr`. ✅

In [8]:
print(np.dot(Q, R))

[[1. 4. 8. 4.]
 [4. 2. 3. 7.]
 [8. 3. 6. 9.]
 [4. 7. 9. 2.]]


&#128310; Describe in words what you find as the dot product of Q and R.  ✅

✍🏽 The dot product of Q and R is equal to our original matrix A, which validates that Q and R are the decomposition pair of the matrix A. When we multiply Q and R back together, we recover the matrix A. 

&#128309; Now let's use our QR decomposition to compute the eigenvalues and eigenvectors of $\mathbf{A}$ via the QR algorithm described above. Implement this following the comments below.  ✅

In [9]:
def QR_algorithm(A, delta=1.0, epsilon=1E-6):
    # Get N, the side length of A
    N, _ = A.shape
    # Use np.identity to create an NxN identity matrix and store it in V
    V = np.identity(N)
    
    while delta > epsilon:
        # Perform a QR decomposition on A using your function QR_decomposition
        Q, R = QR_decomposition(A)
        # Take the dot product of R and Q using np.dot and store the result in A
        A = np.dot(R, Q)
        # Take the dot product of V and Q using np.dot and store the result in V
        V = np.dot(V, Q)
        # Compute the sum of the lower triangular elements of A. You may want to use np.tril
        delta = np.abs(np.sum(np.tril(A, -1)))
        # Delete "pass" when you write real code!
        # pass

    # Return A, V
    return A, V

&#128309; The end result of this code should be that the diagonal elements of $\mathbf{A}$ contain the eigenvalues and the columns of $\mathbf{V}$ contain the eigenvectors. Run the line of code below to apply your algorithm and store the output as `Afinal` and `Vfinal`. ✅ Print each of these nicely. You can also `imshow` your `Afinal` to visually confirm its diagonality.  ✅

In [10]:
Afinal, Vfinal = QR_algorithm(A)

eigenvalues = [Afinal[i][i] for i in range(len(Afinal))]

for i in range(len(Afinal)):
    print(f"λ_{i} = ", np.round(Afinal[i][i], decimals=0), f" and V_{i} = ", Vfinal[i])

print("\n Eigenvalue list A is \n", np.round(eigenvalues, decimals=2))
print("\n Eigenvector Matrix V is \n", Vfinal)

λ_0 =  21.0  and V_0 =  [ 0.43151698 -0.38357064 -0.77459666 -0.25819889]
λ_1 =  -8.0  and V_1 =  [ 0.38357063  0.43151698 -0.2581989   0.77459667]
λ_2 =  -3.0  and V_2 =  [ 0.62330228  0.52740965  0.25819889 -0.51639778]
λ_3 =  1.0  and V_3 =  [ 0.52740965 -0.62330227  0.51639779  0.25819889]

 Eigenvalue list A is 
 [21. -8. -3.  1.]

 Eigenvector Matrix V is 
 [[ 0.43151698 -0.38357064 -0.77459666 -0.25819889]
 [ 0.38357063  0.43151698 -0.2581989   0.77459667]
 [ 0.62330228  0.52740965  0.25819889 -0.51639778]
 [ 0.52740965 -0.62330227  0.51639779  0.25819889]]


&#128309; Use `np.linalg.eig` to solve for the eigenvalues and eigenvectors of `A`. Compare your results. ✅

In [11]:
eigenvalues, eigenvectors = np.linalg.eig(A)

for i in range(len(Afinal)):
    print(f"λ_{i} = ", np.round(eigenvalues[i], decimals=0), f" and V_{i} = ", eigenvectors[i])

print("\n Eigenvalue list A is \n", np.round(eigenvalues, decimals=2))
print("\n Eigenvector Matrix V is \n", eigenvectors)

λ_0 =  21.0  and V_0 =  [ 0.43151697  0.38357064 -0.77459667  0.25819889]
λ_1 =  -8.0  and V_1 =  [ 0.38357064 -0.43151697 -0.25819889 -0.77459667]
λ_2 =  -3.0  and V_2 =  [ 0.62330229 -0.52740963  0.25819889  0.51639778]
λ_3 =  1.0  and V_3 =  [ 0.52740963  0.62330229  0.51639778 -0.25819889]

 Eigenvalue list A is 
 [21. -8. -3.  1.]

 Eigenvector Matrix V is 
 [[ 0.43151697  0.38357064 -0.77459667  0.25819889]
 [ 0.38357064 -0.43151697 -0.25819889 -0.77459667]
 [ 0.62330229 -0.52740963  0.25819889  0.51639778]
 [ 0.52740963  0.62330229  0.51639778 -0.25819889]]


&#128310; Do you get the same answer? ✅

✍🏽 Yes, we got the same answer using either method. This makes sense as both methods solve for the same final value. We can tell that at the final couple of decimals, our method seems to not entirely converge, and this possibly could be because of the choice of epsilon that doesn't fully converge. 

# Acknolwedgments

S.E. Clark 2024, with parts adapted from Newman 2013.